# **Improving Employee Retention By Predicting Employee Attrition Using Machine Learning**

### **Import Libraries and Settings**

In [1]:
# Import initial necessary libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Settings of dataframe display
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = None

# Version requirements
print('numpy version : ',np.__version__)
print('pandas version : ',pd.__version__)
print('seaborn version : ',sns.__version__)

numpy version :  1.26.4
pandas version :  2.2.1
seaborn version :  0.13.2


## Load Dataset

In [2]:
df = pd.read_csv('Dataset CSV Version.csv')

In [3]:
df.sample(5)

,Username,EnterpriseID,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,NomorHP,Email,TingkatPendidikan,PernahBekerja,IkutProgramLOP,AlasanResign,TanggalLahir,TanggalHiring,TanggalPenilaianKaryawan,TanggalResign
146,ferventAbalone7,105357,Belum_menikah,Wanita,FullTime,Product Design (UI & UX),Senior_level,Bagus,Jakarta Timur,Employee_Referral,3,3.0,0.0,0.0,16.0,+6282250643xxx,ferventAbalone7081@proton.com,Sarjana,1,NaN,tidak_bahagia,1966-04-17,2016-7-21,2020-1-14,2018-04-01
57,insecureShads2,111302,Belum_menikah,Wanita,Outsource,Product Design (UI & UX),Freshgraduate_program,Sangat_bagus,Jakarta Utara,LinkedIn,2,2.0,0.0,6.0,3.0,+6289741767xxx,insecureShads2204@proton.com,Sarjana,1,NaN,masih_bekerja,1980-09-30,2014-3-31,2019-03-09,-
220,soreDunbird3,101434,Belum_menikah,Pria,Outsource,Data Engineer,Freshgraduate_program,Sangat_bagus,Jakarta Utara,LinkedIn,4,4.0,0.0,0.0,11.0,+6285615899xxx,soreDunbird3996@hotmail.com,Magister,1,NaN,NaN,1982-07-22,2011-5-16,2016-3-30,-
227,alertSyrup3,111486,Belum_menikah,Pria,FullTime,Software Engineer (Android),Mid_level,Biasa,Jakarta Utara,Google_Search,4,5.0,0.0,0.0,4.0,+6285607737xxx,alertSyrup3002@hotmail.com,Sarjana,1,NaN,NaN,1987-03-18,2011-04-04,2019-07-02,-
133,sheepishWidgeon1,101510,Belum_menikah,Wanita,Outsource,Software Engineer (iOS),Mid_level,Bagus,Jakarta Selatan,Diversity_Job_Fair,1,4.0,0.0,5.0,4.0,+6289832939xxx,sheepishWidgeon1917@outlook.com,Magister,1,NaN,masih_bekerja,1968-06-06,2016-07-04,2020-2-18,-


In [4]:
# Checking shape of dataframe
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns {df.shape[1]}')

Number of rows: 287
Number of columns 25


The original dataframe has 287 rows and 25 columns.

In [5]:
# Dataset overview
desc_col = []

for col in df.columns :
    desc_col.append([col, df[col].dtype, df[col].isna().sum(), round(df[col].isna().sum()/len(df) * 100, 2), df.duplicated().sum(), df[col].nunique(), df[col].unique()[:5]])

desc_df = pd.DataFrame(data=desc_col, columns='Feature, Data Type, Null Values, Null Percentage (%), Duplicated Values, Unique Values, 5 Unique Sample'.split(","))
desc_df

,Feature,Data Type,Null Values,Null Percentage (%),Duplicated Values,Unique Values,5 Unique Sample
0,Username,object,0,0.00,0,285,"[spiritedPorpoise3, jealousGelding2, pluckyMuesli3, stressedTruffle1, shyTermite7]"
1,EnterpriseID,int64,0,0.00,0,287,"[111065, 106080, 106452, 106325, 111171]"
2,StatusPernikahan,object,0,0.00,0,5,"[Belum_menikah, Menikah, Bercerai, Lainnya, -]"
3,JenisKelamin,object,0,0.00,0,2,"[Pria, Wanita]"
4,StatusKepegawaian,object,0,0.00,0,3,"[Outsource, FullTime, Internship]"
5,Pekerjaan,object,0,0.00,0,14,"[Software Engineer (Back End), Data Analyst, Software Engineer (Front End), Product Manager, Software Engineer (Android)]"
6,JenjangKarir,object,0,0.00,0,3,"[Freshgraduate_program, Senior_level, Mid_level]"
7,PerformancePegawai,object,0,0.00,0,5,"[Sangat_bagus, Sangat_kurang, Bagus, Biasa, Kurang]"
8,AsalDaerah,object,0,0.00,0,5,"[Jakarta Timur, Jakarta Utara, Jakarta Pusat, Jakarta Selatan, Jakarta Barat]"
9,HiringPlatform,object,0,0.00,0,9,"[Employee_Referral, Website, Indeed, LinkedIn, CareerBuilder]"


## **About The Dataset**

**Overview:**
- Dataset contains 287 rows, 25 features.

- Dataset consists of 3 data types; float64, int64 and object.

- `TanggalLahir`, `TanggalHiring`, `TanggalPenilaianKaryawan`, `TanggalResign` feature will be changed from object into datetime data type.

- Dataset contains null values in various columns, will be handled after checking the distribution for proper imputation method. As for `IkutProgramLOP` will directly drop since it has high missing percentage. Also some columns that have "-" values will need to be considered as null values and filtered later.

- Some columns like `SkorSurveyEngagement`, `SkorKepuasanPegawai`, `JumlahKeikutsertaanProjek`, `JumlahKeterlambatanSebulanTerakhir`, `JumlahKetidakhadiran`, `IkutProgramLOP` have float data type when it doesn't actually need or representative in decimal value, these feature data type will be changed to integer.

- `PernahBekerja` column have invalid value, it normally should be binary but instead it has 1 and yes. This column will be dropped or the values will need to be re-checked.

- Originally there is no column to determine whether an employee resigned or not, this will be extracted from `TanggalResign` column later.

- (Optional) Changing the name of some columns or the values to standardize the overall writing format might be necessary. For values it probably more prioritized because the format varies so much compared to column names, as for the format of the values will follow `Pekerjaan` column format (ex: Product Manager, Data Analyst, etc)

**Feature Descriptions**

- `Username`: Username of the employee account
- `EnterpriseID`: ID of the employee in the company
- `StatusPernikahan`: Marital status of the employee
- `JenisKelamin`: Gender of the employee
- `StatusKepegawaian`: Employment status of the employee
- `Pekerjaan`: Role of the employee
- `JenjangKarir`: Level of experience of the employee
- `PerformancePegawai`: Employee performance category score
- `AsalDaerah`: Employee region of origin
- `HiringPlatform`: Platform the employee application is accepted

- `SkorSurveyEngagement`: Level of employee engagement within the organization
- `SkorKepuasanPegawai`: Level of how satisfied employees are with their job and the workplace
- `JumlahKeikutsertaanProjek`: Number of times the employee join a project
- `JumlahKeterlambatanSebulanTerakhir`: Number of times the employee is late
- `JumlahKetidakhadiran`: Number of times the employee is absent
- `NomorHP`: Handphone number of the employee
- `Email`: Personal email of the employee
- `TingkatPendidikan`: Education level Handphone number of the employee
- `PernahBekerja`: Whether the employee have previous work experience or not
- `IkutProgramLOP`: Whether the employee join LOP Program or not

- `AlasanResign`: Reason for resignation of the employee
- `TanggalLahir`: Birth date of the employee
- `TanggalHiring`: Hiring date of the employee
- `TanggalPenilaianKaryawan`: Scoring date of the employee
- `TanggalResign`: Resignation date of the employee

## Initial Transformation

In [8]:
df['PernahBekerja'].value_counts()

PernahBekerja
1      286
yes      1
Name: count, dtype: int64

Even if the 'yes' value in `PernahBekerja` column originally should be 0, this column doesn't mean anything. So this column will be drop.

In [11]:
# Drop columns that have invalid and large missing percentage
df.drop(columns=['PernahBekerja', 'IkutProgramLOP'], inplace=True)

In [17]:
# Rename value names in some columns to maintain format similarity
import re

def add_spaces_to_columns(column_names):
    new_column_names = []
    for name in column_names:

        # Use regular expression to add a space before each capital letter, except the first one
        new_name = re.sub(r'(?<!^)(?=[A-Z])', ' ', name)
        new_column_names.append(new_name)
    return new_column_names

# Make list of columns
columns = df.columns

# Process the list
formatted_columns = add_spaces_to_columns(columns)
formatted_columns

def snake_to_title(column_names):
    new_column_names = []
    for name in column_names:
        # Split the name by underscores and capitalize each word
        words = name.split('_')
        capitalized_words = [word.capitalize() for word in words]
        # Join the words with spaces
        new_name = ' '.join(capitalized_words)
        new_column_names.append(new_name)
    return new_column_names

# List of columns in snake_case
columns = ['masih_bekerja', 'toxic_culture', 'jam_kerja', 'ganti_karir', 'tidak_bahagia']

# Process the list
formatted_columns = snake_to_title(columns)
formatted_columns

In [13]:
?re.sub

Signature: re.sub(pattern, repl, string, count=0, flags=0)
Docstring:
Return the string obtained by replacing the leftmost
non-overlapping occurrences of the pattern in string by the
replacement repl.  repl can be either a string or a callable;
if a string, backslash escapes in it are processed.  If it is
a callable, it's passed the Match object and must return
a replacement string to be used.
File:      c:\users\cikal merdeka\appdata\local\programs\python\python312\lib\re\__init__.py
Type:      function

In [13]:
df.columns

Index(['Username', 'EnterpriseID', 'StatusPernikahan', 'JenisKelamin',
       'StatusKepegawaian', 'Pekerjaan', 'JenjangKarir', 'PerformancePegawai',
       'AsalDaerah', 'HiringPlatform', 'SkorSurveyEngagement',
       'SkorKepuasanPegawai', 'JumlahKeikutsertaanProjek',
       'JumlahKeterlambatanSebulanTerakhir', 'JumlahKetidakhadiran', 'NomorHP',
       'Email', 'TingkatPendidikan', 'PernahBekerja', 'IkutProgramLOP',
       'AlasanResign', 'TanggalLahir', 'TanggalHiring',
       'TanggalPenilaianKaryawan', 'TanggalResign'],
      dtype='object')

In [ ]:
# Grouping columns based on data types
nums_cols = ['Username', 'EnterpriseID', 'StatusPernikahan', 'JenisKelamin',
            'StatusKepegawaian', 'Pekerjaan', 'JenjangKarir', 'PerformancePegawai',
            'AsalDaerah', 'HiringPlatform', 'SkorSurveyEngagement', 'SkorKepuasanPegawai', 'NomorHP',
            'Email', 'TingkatPendidikan', 'PernahBekerja', 'IkutProgramLOP', 'AlasanResign']

cats_cols = ['JumlahKeikutsertaanProjek', 'JumlahKeterlambatanSebulanTerakhir', 'JumlahKetidakhadiran']

date_cols = ['TanggalLahir', 'TanggalHiring', 'TanggalPenilaianKaryawan', 'TanggalResign']

In [41]:
check_type_cat_num = list(df['JumlahKetidakhadiran'].value_counts())
print(check_type_cat_num)

[20, 20, 19, 17, 17, 16, 16, 16, 15, 15, 14, 14, 13, 12, 12, 10, 10, 9, 7, 7, 1, 1]


- Categorical columns (16 columns): 
    - `Username` : nominal
    - `EnterpriseID` : nominal
    - `StatusPernikahan` : nominal
    - `JenisKelamin` : nominal
    - `StatusKepegawaian` : nominal
    - `Pekerjaan` : nominal
    - `JenjangKarir` : ordinal
    - `PerformancePegawai` : ordinal
    - `AsalDaerah` : nominal
    - `HiringPlatform` : nominal
    - `SkorSurveyEngagement` : ordinal
    - `SkorKepuasanPegawai` : ordinal
    - `NomorHP` : nominal
    - `Email` : nominal
    - `TingkatPendidikan` : ordinal
    - `AlasanResign` : nominal

- Numerical/Continuous columns (3 columns) :

    - `JumlahKeterlambatanSebulanTerakhir, JumlahKetidakhadiran, JumlahKeikutsertaanProjek`

- Date column (4 column) :
    -  `TanggalLahir, TanggalHiring, TanggalPenilaianKaryawan, TanggalResign`

In [ ]:
df['AlasanResign'].value_counts()

AlasanResign
masih_bekerja               132
jam_kerja                    16
ganti_karir                  14
kejelasan_karir              11
tidak_bisa_remote            11
toxic_culture                10
leadership                    9
tidak_bahagia                 8
internal_conflict             4
Product Design (UI & UX)      4
apresiasi                     2
Name: count, dtype: int64

In [ ]:
df[['AlasanResign', 'TanggalResign']].sample(10)

,AlasanResign,TanggalResign
145,masih_bekerja,-
115,masih_bekerja,-
136,tidak_bahagia,2015-04-01
4,ganti_karir,2018-09-06
277,NaN,-
243,NaN,-
208,NaN,-
123,masih_bekerja,-
130,masih_bekerja,-
58,masih_bekerja,-
